In [1]:
import transformers
print(transformers.__version__)


/root/anaconda3/envs/kvcache/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.38.0


In [2]:
import copy
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig
from modify_llama_helm import convert_kvcache_llama_heavy_recent
import torch
import time
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
prompt_text = 'In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity.'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

######## Generate with Full Cache
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
print(model.state_dict())
checkpoint = copy.deepcopy(model.state_dict())
model.eval().to(torch.device("cuda:2"))

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


OrderedDict([('model.embed_tokens.weight', tensor([[ 1.1921e-06, -1.7881e-06, -4.2915e-06,  ...,  8.3447e-07,
         -6.4373e-06,  8.9407e-07],
        [ 1.8387e-03, -3.8147e-03,  9.6130e-04,  ..., -9.0332e-03,
          2.6550e-03, -3.7537e-03],
        [ 1.0193e-02,  9.7656e-03, -5.2795e-03,  ...,  2.9297e-03,
          4.0817e-04, -5.0964e-03],
        ...,
        [-1.3550e-02, -3.5095e-03, -1.8921e-02,  ..., -9.3384e-03,
          8.7891e-03, -1.2741e-03],
        [-1.0681e-02,  8.9722e-03,  1.2573e-02,  ..., -3.3691e-02,
         -1.6235e-02,  3.0212e-03],
        [-9.0942e-03, -1.8082e-03, -6.9809e-04,  ...,  3.8452e-03,
         -1.2085e-02,  7.2861e-04]], dtype=torch.float16)), ('model.layers.0.self_attn.q_proj.weight', tensor([[-0.0060, -0.0146, -0.0021,  ...,  0.0042,  0.0018, -0.0035],
        [ 0.0142, -0.0043,  0.0032,  ..., -0.0092, -0.0108,  0.0073],
        [-0.0137,  0.0121,  0.0002,  ...,  0.0061,  0.0181, -0.0030],
        ...,
        [ 0.0018,  0.0093, -0.0006, 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [3]:
# input_ids = tokenizer(prompt_text, return_tensors='pt').input_ids.to(model.device)
input_ids = tokenizer(prompt_text, add_special_tokens=False, return_tensors='pt').input_ids.to(torch.device("cuda:2"))
generate_ids = model.generate(input_ids, max_new_tokens=64)
result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print("################## Generated Context with Full Cache ###################")
print(result)

################## Generated Context with Full Cache ###################
In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity. Without warning, he began to strum a haunting melody, captivating the audience with his soulful voice and infectious energy. As the music poured forth, a deep sense of connection and belonging was palpable in the room, transcending the boundaries of age, race, and


In [4]:
print("loading checkpoint")
# time.sleep(5)
config = AutoConfig.from_pretrained(model_path)
config.heavy_ratio = 0.1
config.recent_ratio = 0.1

loading checkpoint


In [5]:
model_new = convert_kvcache_llama_heavy_recent(model, config)

# print(model_new.state_dict())


In [6]:
model_new.state_dict()

OrderedDict([('model.embed_tokens.weight',
              tensor([[ 1.1921e-06, -1.7881e-06, -4.2915e-06,  ...,  8.3447e-07,
                       -6.4373e-06,  8.9407e-07],
                      [ 1.8387e-03, -3.8147e-03,  9.6130e-04,  ..., -9.0332e-03,
                        2.6550e-03, -3.7537e-03],
                      [ 1.0193e-02,  9.7656e-03, -5.2795e-03,  ...,  2.9297e-03,
                        4.0817e-04, -5.0964e-03],
                      ...,
                      [-1.3550e-02, -3.5095e-03, -1.8921e-02,  ..., -9.3384e-03,
                        8.7891e-03, -1.2741e-03],
                      [-1.0681e-02,  8.9722e-03,  1.2573e-02,  ..., -3.3691e-02,
                       -1.6235e-02,  3.0212e-03],
                      [-9.0942e-03, -1.8082e-03, -6.9809e-04,  ...,  3.8452e-03,
                       -1.2085e-02,  7.2861e-04]], device='cuda:2', dtype=torch.float16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0118,  0.0144, -0.014

In [7]:
model_new.load_state_dict(checkpoint)
model_new.half().eval().to(torch.device("cuda:2"))

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention_heavy_hitter(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (

In [8]:
prompt_text_new = 'what is the meaning of life?'
generate_ids_hh = tokenizer.encode(prompt_text_new, return_tensors='pt').to(torch.device("cuda:2")) 

In [12]:
model_new.model.layers

ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention_heavy_hitter(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [10]:
generate_ids_hh

tensor([[    1,   825,   338,   278,  6593,   310,  2834, 29973]],
       device='cuda:2')

In [11]:
print("loading new model")
generate_ids_hh = model_new.generate(generate_ids_hh, max_new_tokens=64,use_cache=True)

loading new model


KeyError: 'Cache only has 0 layers, attempted to access layer with index 0'

In [ ]:
# result_hh = tokenizer.batch_decode(generate_ids_hh, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# print("################## Generated Context with Heavy Hitter Oracle ###################")
# print(result_hh)